# Final Project - Part 2

## 1. Title: “CareerChat: Your AI Job Outreach Assistant”

Team:
1.   Anshika Bajpai
2.   Brendan Kelly
3.   Cassie Cagwin

11/9/2025

## 2. Preprocessing 30pts

---

Provide all essential steps that you deem necessary for your application

In [4]:
##Ask if they'd like to upload their resume or fill out a form with their profile
profile_creation_method = 'Yes, I would like to upload my resume.' #@param ['Yes, I would like to upload my resume.', 'No, I would like to fill out the profile form.']
print(f"Selection: {profile_creation_method}")

Selection: Yes, I would like to upload my resume.


In [5]:
##Profile info imports
##Name, work experience, education, skills, career interest
import ipywidgets as widgets
from IPython.display import display, clear_output

if profile_creation_method == 'No, I would like to fill out the profile form.':

  name_input = widgets.Text(
      description='Name:',
      placeholder='Enter your full name',
      layout=widgets.Layout(width='auto')
  )


  email_input = widgets.Text(
      description='Email:',
      placeholder='user@example.com',
      layout=widgets.Layout(width='auto')
  )

  experience_input = widgets.Textarea(
      description='Work Experience:',
      placeholder='List companies, roles, and achievements (one per line)',
      rows=5, # Number of visible rows
      layout=widgets.Layout(width='auto')
  )

  education_input = widgets.Textarea(
      description='Education:',
      placeholder='List degrees, institutions, and dates',
      rows=3,
      layout=widgets.Layout(width='auto')
  )

  skills_input = widgets.Textarea(
      description='Skills:',
      placeholder='List your key skills (e.g., Python, SQL, NLP)',
      rows=3,
      layout=widgets.Layout(width='auto')
  )

  interest_input = widgets.Textarea(
      description='Career Interests:',
      placeholder='Please describe your career interests',
      rows=3,
      layout=widgets.Layout(width='auto')
  )

  submit_button = widgets.Button(description="Submit Profile", button_style='success')
  output_area = widgets.Output()

  def on_submit_click(b):
      with output_area:
          clear_output(wait=True)

          print("Profile Information")
          print(f"Name: {name_input.value}")
          print(f"Email: {email_input.value}")
          print("Career Interests:")
          print(interest_input.value)
          print("Work Experience:")
          print(experience_input.value)
          print("\nEducation:")
          print(education_input.value)
          print("\nSkills:")
          print(skills_input.value)

  submit_button.on_click(on_submit_click)

  form_widgets = [
      name_input,
      email_input,
      widgets.VBox([
          widgets.HTML("<b>Career Interests:</b>"),
          interest_input
      ]),
      widgets.VBox([
          widgets.HTML("<b>Work Experience (Details):</b>"),
          experience_input
      ]),
      widgets.VBox([
          widgets.HTML("<b>Education (Details):</b>"),
          education_input
      ]),
      widgets.VBox([
          widgets.HTML("<b>Skills (Keywords):</b>"),
          skills_input
      ]),
      submit_button,
      output_area
  ]

  full_form = widgets.VBox(form_widgets)
  display(full_form)


ModuleNotFoundError: No module named 'ipywidgets'

In [ ]:
# ##Resume upload
# from google.colab import files
# if profile_creation_method == 'Yes, I would like to upload my resume.':
#     print("Please upload your resume.")
#     resume = files.upload()

#     for fn in resume.keys():
#         print('User uploaded file "{name}" with length {length} bytes'.format(
#             name=fn, length=len(resume[fn])))

In [ ]:
##Resume parsing
##PDF only for now?



In [ ]:
##Company
company = input("What company are you messaging about? ")

##Role type
role_type = input("What type of role are you messaging about? ")

##Job title
job_title = input("What is the job title you are messaging about? ")

##Job description
job_description = input("What is the job description you are messaging about? ")

In [ ]:
##Message type selection
##Options: LinkedIn connection notes, InMails, Referral requests, Cover letters, Thank-you notes, Recruiter outreach emails
import ipywidgets as widgets
from IPython.display import display

menu_options = [
    'LinkedIn connection notes',
    'InMails',
    'Referral requests',
    'Cover letters',
    'Thank-you notes',
    'Recruiter outreach emails'
]

message_type_dropdown = widgets.Dropdown(
    options=menu_options,
    value= 'LinkedIn connection notes',
    description='Choose the type of message you want to write:',
    disabled=False,
)

display(message_type_dropdown)

selected_message_type = message_type_dropdown.value

In [ ]:
##Optional context: tone, structure, content
tone_description = input("Please specify any particular tone you want the message to have.")
structure_description = input("Please specify any particular structure you want the message to have.")
content_description = input("Please specify any particular content you want the message to have.")

## 3. Feature Extraction 30pts

---

Implement any existing feature extraction tools and methods (term frequency, word embeddings etc)

In [1]:
!pip install python-dateutil newspaper3k GoogleNews pandas lxml_html_clean nltk transformers

In [1]:
import time
import random
import requests
import pandas as pd
import nltk
nltk.download('punkt')
from datetime import datetime
from dateutil.relativedelta import relativedelta
from datetime import datetime
from dateutil.relativedelta import relativedelta
from newspaper import Article, Config
from GoogleNews import GoogleNews
from nltk.tokenize import sent_tokenize
from transformers import pipeline
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter

[nltk_data] Downloading package punkt to C:\Users\New
[nltk_data]     User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
C:\Users\New User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# configuration
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
config = Config()
config.browser_user_agent = user_agent
config.request_timeout = 30
company = "Microsoft" #TODO replace with dynamic company input
role = "Software Engineer" #TODO replace with dynamic role input

# search parameters 
company_query = f"{company} Latest News"
end_date_str = datetime.now().strftime('%m/%d/%Y')
start_date_str = (datetime.now() - relativedelta(months=3)).strftime('%m/%d/%Y')

# establish request session
session = requests.Session()
retries = Retry(total=5, backoff_factor=1)
adapter = HTTPAdapter(max_retries=retries)
session.mount('http://', adapter)
session.mount('https://', adapter)

# search 5 pages of Google News results for the company's latest news
gn = GoogleNews(start=start_date_str, end=end_date_str)
gn.search(company_query)
pages_to_fetch = 5

news_items = []
seen_links = []
for page in range(1, pages_to_fetch + 1):
    gn.getpage(page)
    items = gn.results()
    if not items:
        continue
    for item in items:
        link = item.get('link')
        if not link or link in seen_links:
            continue
        seen_links.append(link)
        news_items.append(item)

# iterate over each article and parse using newspaper library
parsed_articles = []
failed_articles = []

for i, item in enumerate(news_items):
    article_link = item.get('link')
    
    headers = {
        "User-Agent": user_agent,
        "Accept-Language": "en-US,en;q=0.9",
        "Referer": "https://www.google.com/"
    }

    try:
        # get HTML content with retries
        resp = session.get(article_link, headers=headers, timeout=5)
        resp.raise_for_status()
        html = resp.text

        # give the HTML to newspaper to avoid its default downloader being blocked
        article = Article(article_link, config=config)
        article.download(input_html=html)
        article.parse()
        article.nlp()
        if not article.text and not article.title:
            continue

        parsed_articles.append({
            'Title': article.title,
            'Published': item.get('date', 'N/A'),
            'Source': item.get('media', 'N/A'),
            'Link': article_link,
            'Summary': article.summary,
            'Full_Text': article.text
        })

    except Exception as e:
        # store failed articles in separate list
        failed_articles.append({
            'Title': item.get('title', 'N/A'),
            'Published': item.get('date', 'N/A'),
            'Source': item.get('media', 'N/A'),
            'Link': article_link,
            'Summary': "Scraping failed.",
            'Full_Text': f"Error during parsing/download: {e}"
        })

    # add a random pause between 2 and 6 seconds to mitigate blocks/throttling 
    time_to_wait = random.uniform(2, 6)
    time.sleep(time_to_wait)

print("Articles parsed: ", len(parsed_articles))
df = pd.DataFrame(parsed_articles)
df.head()

Articles parsed:  17


,Title,Published,Source,Link,Summary,Full_Text
0,My new favorite deep Windows lore: Microsoft o...,7 hours ago,PC Gamer,https://www.pcgamer.com/hardware/microsoft-win...,I recently learned about a perfect example of ...,There's something I find deeply funny about th...
1,Assessing Microsoft’s Value After Recent 4.1% ...,8 hours ago,Yahoo Finance,https://finance.yahoo.com/news/assessing-micro...,"For mature, profitable companies like Microsof...","For mature, profitable companies like Microsof..."
2,My new favorite deep Windows lore: Microsoft o...,7 hours ago,PC Gamer,https://www.pcgamer.com/hardware/microsoft-win...,I recently learned about a perfect example of ...,There's something I find deeply funny about th...
3,Assessing Microsoft’s Value After Recent 4.1% ...,8 hours ago,Yahoo Finance,https://finance.yahoo.com/news/assessing-micro...,"For mature, profitable companies like Microsof...","For mature, profitable companies like Microsof..."
4,Malicious AI-made extension with ransomware ca...,2 hours ago,TechRadar,https://www.techradar.com/pro/security/malicio...,Malicious VS Code extension ‘susvsex’ acted as...,Malicious VS Code extension ‘susvsex’ acted as...


In [3]:
summarizer = pipeline("text2text-generation", model="google/flan-t5-large")

# summarize articles
summaries = []

for article in parsed_articles:
    text = article.get('Full_Text').strip()
    if not text:
        summaries.append((article.get('Link'),'Summary failure.'))

    article_sentences = sent_tokenize(text)
    
    prompt = f"Write a one sentence summary of the content contained in the following article; specifically highlight its impact or relevance on {company} in a context that would be useful for a job interview of a position as a {role} at the company:\n{text}"

    summary_result = summarizer(prompt,max_length=130)

    summaries.append((article.get('Link'), summary_result[0].get('generated_text')))


Device set to use cpu
Token indices sequence length is longer than the specified maximum sequence length for this model (1311 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
print(summaries)

In [ ]:
# old scraping code
# configuration
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
config = Config()
config.browser_user_agent = user_agent
config.request_timeout = 30
company = "Microsoft" #TODO replace with dynamic company input
role = "Software Engineer" #TODO replace with dynamic role input

# search parameters 
company_query = f"{company} Latest News"
end_date_str = datetime.now().strftime('%m/%d/%Y')
start_date_str = (datetime.now() - relativedelta(months=3)).strftime('%m/%d/%Y')

gn = GoogleNews(start=start_date_str, end=end_date_str)
gn.search(company_query)
news_items = gn.results()
parsed_articles = []

# loop over news articles
for i, item in enumerate(news_items):
    article_link = item.get('link')

    try:
        article = Article(article_link, config=config)

        article.download()
        article.parse()
        article.nlp()

        if not article.text and not article.title:
            continue

        parsed_articles.append({
            'Title': article.title,
            'Published': item.get('date', 'N/A'),
            'Source': item.get('media', 'N/A'),
            'Link': article_link,
            'Summary': article.summary,
            'Full_Text': article.text
        })

    except Exception as e:
        # TODO refine error handling so that we don't stop until we have 10 articles (e.g. failed article doesn't count)
        parsed_articles.append({
            'Title': item.get('title', 'N/A'),
            'Published': item.get('date', 'N/A'),
            'Source': item.get('media', 'N/A'),
            'Link': article_link,
            'Summary': "Scraping failed.",
            'Full_Text': f"Error during parsing/download: {e}"
        })

    # add a random pause between 2 and 6 seconds to mitigate blocks/throttling 
    time_to_wait = random.uniform(2, 6)
    time.sleep(time_to_wait)

df = pd.DataFrame(parsed_articles)
df.head()

## 4. Main Functionality 10pts

---

This is your main task. For example, if you are creating a translation app, your main task is Translation.

In [ ]:
import time
import random
from datetime import datetime
from dateutil.relativedelta import relativedelta
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
from newspaper import Article, Config
from GoogleNews import GoogleNews # Use the alternative package

# --- Configuration (Keep robust settings) ---
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
config = Config()
config.browser_user_agent = user_agent
config.request_timeout = 30
config.memoize_articles = False # Avoid cache issues
company = "Microsoft" #TODO replace with dynamic company input
role = "Software Engineer" #TODO replace with dynamic role input

# --- Setup Search Parameters (assuming company is defined) ---
company_query = f"{company} Latest News"
end_date_str = datetime.now().strftime('%m/%d/%Y')
start_date_str = (datetime.now() - relativedelta(months=3)).strftime('%m/%d/%Y')

gn = GoogleNews(start=start_date_str, end=end_date_str)
gn.search(company_query)
news_items = gn.results()
parsed_articles = []

print(f"Starting scrape for {len(news_items)} articles. This will take a few minutes...")

# --- Loop with Sleep ---
for i, item in enumerate(news_items):
    article_link = item.get('link')

    try:
        article = Article(article_link, config=config)

        article.download()
        article.parse()
        article.nlp()

        if not article.text and not article.title:
            raise Exception("Article text and title were blank after parsing.")

        parsed_articles.append({
            'Title': article.title,
            'Published': item.get('date', 'N/A'),
            'Source': item.get('media', 'N/A'),
            'Link': article_link,
            'Summary': article.summary,
            'Full_Text': article.text
        })

    except Exception as e:
        # TODO refine error handling so that we don't stop until we have 10 articles (e.g. failed article doesn't count)
        parsed_articles.append({
            'Title': item.get('title', 'N/A'),
            'Published': item.get('date', 'N/A'),
            'Source': item.get('media', 'N/A'),
            'Link': article_link,
            'Summary': "Scraping failed.",
            'Full_Text': f"Error during parsing/download: {e}"
        })

    # Add a random pause between 2 and 6 seconds
    # This is CRITICAL for avoiding blocks/throttling
    time_to_wait = random.uniform(2, 6)
    time.sleep(time_to_wait)

    if (i + 1) % 10 == 0:
        print(f"Processed {i + 1} articles. Waiting...")

df = pd.DataFrame(parsed_articles)
df.head()

Starting scrape for 10 articles. This will take a few minutes...
Processed 10 articles. Waiting...


,Title,Published,Source,Link,Summary,Full_Text
0,Microsoft Embraces AI-Driven Hiring Spree: Nad...,1 hour ago,OpenTools,https://opentools.ai/news/microsoft-embraces-a...,Scraping failed.,Error during parsing/download: partially initi...
1,Microsoft is making it easier for Windows 11 u...,2 hours ago,The Times of India,https://timesofindia.indiatimes.com/technology...,Scraping failed.,Error during parsing/download: Article `downlo...
2,Microsoft Weekly: the redesigned Start menu an...,2 hours ago,Neowin,https://www.neowin.net/news/microsoft-weekly-t...,Scraping failed.,Error during parsing/download: Article `downlo...
3,Microsoft Plots AI-Powered Hiring Surge After ...,4 hours ago,Republic World,https://www.republicworld.com/business/microso...,Scraping failed.,Error during parsing/download: Article `downlo...
4,"Microsoft 365 Copilot drops ""AI can make mista...",4 hours ago,Windows Latest,https://www.windowslatest.com/2025/11/02/micro...,Scraping failed.,Error during parsing/download: Article `downlo...


In [ ]:

nltk.download('punkt')

from nltk.tokenize import sent_tokenize
from transformers import pipeline

# TODO pick pipeline model
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

##Summarize articles
summaries = []

for article in parsed_articles:
    text = article.get('Full_Text').strip()
    if not text:
        summaries.append((article.get('Link'),'Summary failure.'))

    article_sentences = sent_tokenize(text)
    
    prompt = f"Write a one sentence summary for the following article, specifically related to its impact or relevance on {company} in a context that may be useful for a job interview of a position as a {role}:\n{text}"

    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

    summary_result = summarizer(prompt)
    print(summary_result)
    print(summary_result[0])
    print(summary_result[0].get('summary_text'))

    if '.' in summary_result['summary_text']:
        summaries.append((article.get('Link'), summary_result[0].get('summary_text').split('. ')))
    else:
        summaries.append((article.get('Link'), summary_result[0].get('summary_text')))

print(summaries)

AttributeError: partially initialized module 'nltk' has no attribute 'data' (most likely due to a circular import)

In [ ]:
##Generate message


## 5. Personal Contribution Statement 10pts

---

*   Summary of tasks and team members' contributions
*   Proofreading




**Our assignments for Part 2...**

1. PDF parsing + turn into profile - **Cassie**
	* Offer parsed resume as a form where they can correct values
2. News scraping (+ Reddit https://www.reddit.com/dev/api/) + summarization - **Brendan**
	* Offer filters for topics in news
3. Prompted engineering for messages (including offering news highlights to choose from + maybe reddit) + code to generate message - **Anshika**
	* Who to send to form: Who? How do you know them?
	* Creativity (temperature) measure?

### Main Functionality

This project focuses on building a personalized message-generation assistant using prompt engineering to help users craft engaging outreach messages. The system will allow users to specify the recipient (name, relationship, and context of how they know them), and will incorporate customizable creativity levels through a temperature setting to control tone and originality. It will offer curated content options: such as recent news highlights, Reddit threads, or trending topics allowing the user to select relevant hooks to include in the message. The tool will then generate a tailored, well-crafted message using optimized prompts and optional code-based automation for message creation, ensuring personalization, relevance, and creativity in communication.

In [3]:
#Import libraries
import json
import os
from jinja2 import Environment, FileSystemLoader
from dotenv import load_dotenv 
import openai

In [4]:
#Load OpenAI API key from .env file
load_dotenv()
OPENAI_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_KEY:
    raise ValueError("OPENAI_API_KEY not found in environment variables.")

openai.api_key = OPENAI_KEY



In [5]:
#Load message payload template
with open("/Users/anshikabajpai/Desktop/nlp/project/CareerChat/payload.json","r", encoding="utf-8") as f:
    payload = json.load(f)

In [18]:
#Set user varibales (We will use streamlit to get this input later on)
role = 'recruiter'
history = ['indiana university', 'society of women engineers conference 2025']
position = 'machine learning engineer'
message_type = 'LinkedIn connection notes' #'LinkedIn connection notes' # 'Cover Letters'

In [19]:
#News articles about the company | Brendan's edit starts here
news = [
    ('https://www.foxnews.com/tech/windows-10-users-face-ransomware-nightmare-microsoft-support-ends-2025-worldwide', 'Microsoft warns that over 90% of ransomware attacks target unsupported Windows 10 systems, urging users to upgrade ahead of support ending worldwide in 2025.')
    , ('https://www.cnbc.com/2025/11/01/meta-alphabet-amazon-microsoft-earnings-ads.html', 'Microsoft (alongside Meta, Alphabet and Amazon) reported ad- and cloud-driven earnings, reinforcing its expanding role in AI/ads environments.')
    , ('https://www.cnbc.com/2025/10/31/tech-ai-google-meta-amazon-microsoft-spend.html', 'Microsoft signalled a continuing ramp-up in AI infrastructure investment, matching peers in increasing data center and AI-capex spending.')
    , ('https://finance.yahoo.com/news/jim-cramer-says-microsoft-reported-134117610.html', 'Cramer commented that although Microsoft delivered a strong quarter, its stock was still hit partly because of investor concern over its aggressive investment pace.')
    , ('https://www.testingcatalog.com/microsoft-to-broaden-copilot-portraits-with-new-use-cases/', 'Microsoft is expanding its Copilot Portraits feature (AI-avatar tool) into new use cases like public speaking and job-prep modelling.')
    , ('https://www.forbes.com/sites/daveywinder/2025/11/01/new-warning-as-microsoft-windows-attacks-confirmed---no-fix-available/', 'Microsoft acknowledged ongoing attacks exploiting a Windows vulnerability (CVE-2025-9491) with no fix yet available, raising urgent security concerns.')
    , ('https://timesofindia.indiatimes.com/technology/tech-news/100000-tech-layoffs-in-2025-amazon-microsoft-intel-and-these-companies-cut-thousands-of-jobs/articleshow/125015287.cms', 'Microsoft is among major tech firms cutting thousands of jobs (9,000 reportedly) in 2025 amid restructuring and shifting toward AI-driven operations.')
]


In [32]:
#Load Jinja2 template and render prompt
env = Environment(loader=FileSystemLoader('/Users/anshikabajpai/Desktop/nlp/project/CareerChat/'))

# template=env.get_template("template.j2")

if message_type == 'LinkedIn connection notes':
    template = env.get_template("linkedin_msg.j2")
elif message_type == 'Cover Letters':
    template = env.get_template("cover_letter.j2")

prompt = template.render(payload=payload,role=role, history=history, position=position, news=news, company_name="Microsoft", Recipient_name="Alex Johnson")
print(prompt)


You are given below JSON payload:
{
    "education": [
        {
            "degree": "MS",
            "field": "Data Science",
            "institution": "Indiana University"
        },
        {
            "degree": "BS",
            "field": "Computer Science",
            "institution": "University of Illinois"
        }
    ],
    "email": "alex.rivera@example.com",
    "name": "Alexandra Rivera",
    "skills": [
        "Python",
        "Pandas",
        "NumPy",
        "Scikit-learn",
        "PyTorch",
        "TensorFlow",
        "Natural Language Processing",
        "Deep Learning",
        "Data Visualization",
        "SQL",
        "Docker",
        "Kubernetes",
        "AWS",
        "Git"
    ],
    "work_experience": [
        {
            "achievements": [
                "Led a team of 5 in developing NLP models to detect financial fraud from customer communication data.",
                "Deployed end-to-end machine learning pipelines using AWS SageMaker and

In [33]:
#Generate message using OpenAI API
response = openai.ChatCompletion.create(
    model="gpt-4.1-mini", #"gpt-4o-mini",
    messages=[{
        "role": "system",
        "content": "You are a helpful assistant that helps people draft {message_type} based on their background and the job description."
    }, {
        "role": "user",
        "content": prompt
    }],
    max_tokens=512,
    temperature=0.2
)
print(response)





{
  "id": "chatcmpl-Ca20m6OinMysyZCdkzJ8aFG3sSK9u",
  "object": "chat.completion",
  "created": 1762703576,
  "model": "gpt-4.1-mini-2025-04-14",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Hi Alex, great connecting again after Indiana U and SWE 2025! With my ML engineering experience at NeuralEdge and Innova, plus skills in PyTorch, Kubernetes, and AWS, I\u2019m excited about the Machine Learning Engineer role at Microsoft. Would love to discuss how I can contribute.",
        "refusal": null,
        "annotations": []
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 845,
    "completion_tokens": 59,
    "total_tokens": 904,
    "prompt_tokens_details": {
      "cached_tokens": 0,
      "audio_tokens": 0
    },
    "completion_tokens_details": {
      "reasoning_tokens": 0,
      "audio_tokens": 0,
      "accepted_prediction_tokens": 0,
      "rejected_prediction_t

In [34]:
reply = response['choices'][0]['message']['content']
print("Generated Email Draft:\n", reply)

Generated Email Draft:
 Hi Alex, great connecting again after Indiana U and SWE 2025! With my ML engineering experience at NeuralEdge and Innova, plus skills in PyTorch, Kubernetes, and AWS, I’m excited about the Machine Learning Engineer role at Microsoft. Would love to discuss how I can contribute.


### Personal Contribution Statement (10 pts)

**Summary of Tasks and Team Members' Contributions**

- **Cassie** 
- **Brendan** 
- **Anshika** contributed to the core idea and system architecture, and worked on designing the prompt engineering workflow for personalized message generation. This included integrating recipient context (who the message is for and how the user knows them), providing selectable news/Reddit highlights as hooks, and incorporating a creativity control using temperature settings. Anshika also contributed to the code for automated message generation.  

**Proofreading**  
All team members participated in proofreading to ensure clarity, correctness, and cohesiveness across documentation and outputs.


***** Don't forget to put in your specific feedback! *****